In [ ]:
# -*- coding: utf-8 -*-
"""Copy of RagBot.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1zf_uEREmFg7h42VgO_PIdr0GJcU1A_rs
"""


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate
import json



In [ ]:

#Loading ramayana dataset (JSON)
def load_json():
  with open("/content/drive/MyDrive/Colab Notebooks/Datasets/ramayana.json",'r') as f:
    data = json.load(f)
  return data
print(load_json()[1])


In [ ]:

# #Create Chunks
# def create_chunk(dataset):
#   text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10)
#   dataset = Document(page_content=dataset)
#   text_chunk = text_splitter.split_documents([dataset])
#   return text_chunk
# print(create_chunk(load_json()[1]['Translation']))

data = load_json()


In [ ]:

# Create documents
def create_document(data):
    return [Document(page_content=item['Translation']) for item in data]

document = create_document(data)
print(len(document))

# Get embedding model
def get_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

model = get_embedding_model()



In [ ]:
batch_size = 1000
all_embeddings = []

for i in range(0, len(document), batch_size):
    batch_docs = document[i:i+batch_size]
    texts = [doc.page_content for doc in batch_docs]

    batch_embeds = model.embed_documents(texts)

    all_embeddings.extend(batch_embeds)

    print(f"Processed {min(i + batch_size, len(document))} / {len(document)} documents")

print("All embeddings done! Building FAISS index...")



In [ ]:
import pickle


with open("/content/drive/MyDrive/Rag Chat/embeddings.pkl", "wb") as f:
    pickle.dump(all_embeddings, f)
